## pg_server_cache 
Looking on the cache of a PG Server


# Configuration
Open a connection to the DB. Use a Connection String stored in a .cfg file

In [1]:
import sqlalchemy
import pandas as pd
import configparser
import matplotlib.pyplot as plt 

# Read from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 

engine = sqlalchemy.create_engine(con_str)

# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)



# Pre-Req
Check whether the extension exists
TODO: The cell below doesn't really work. The query returns "false" but the code ignores it. Maybe it returns the string "false" rather than a bit

In [9]:

sql_command = """
SELECT CASE WHEN COUNT(*) > 0 THEN B'1'::BIT(1) ELSE B'0'::BIT(1) END AS extension_exists
FROM pg_extension
WHERE extname = 'pg_buffercache';
"""
df = pd.read_sql_query(sql_command, connection)


from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

try:
    # Execute the SQL command
   
    result = connection.execute(sql_command).fetchone()

    # print (type(result[0]))
        # Check the result
    if result[0] == '0':
        raise ValueError("Error: The extension pg_buffercache doesn't exist ")
    else: 
        print ("The Extension pg_buffercache exists")
except (SQLAlchemyError, ValueError) as e:
    # Handle any errors or raised exceptions
    raise e


ValueError: Error: The extension pg_buffercache doesn't exist 

## BUFFER CACHE
Postgres caches recent queries in memory called the shared buffer cache (shared_buffers in postgresql.conf). The pg_statio_user_tables has as rows representing various stats on each of the (user) tables. The two columns of interest we’ll be looking at are:

* pg_statio_user_tables.heap_blks_read — Number of disk blocks read from a table (ie. missed cache)
* pg_statio_user_tables.heap_blks_hits — Number of buffer hits from this table (ie. cache hit)


In [3]:
query_ccnnections_metrics = """
SELECT 
  schemaname, 
  pg_class.relname, 
  ROUND(
    CASE 
      WHEN heap_blks_hit + heap_blks_read = 0 
      THEN 0 
      ELSE heap_blks_hit::decimal / (heap_blks_hit + heap_blks_read) 
    END, 
    3
  ) as cache_hit_ratio
FROM 
  pg_statio_user_tables 
  JOIN pg_class ON pg_statio_user_tables.relid = pg_class.oid 
  JOIN pg_namespace ON pg_namespace.oid = pg_class.relnamespace;
"""

df = pd.read_sql_query(query_ccnnections_metrics, connection)
df

,schemaname,relname,cache_hit_ratio
0,postgres_air,booking_leg,0.420
1,postgres_air,airport,0.933
2,postgres_air,account,0.999
3,postgres_air,frequent_flyer,1.000
4,postgres_air,booking,0.309
5,postgres_air,passenger,0.822
6,postgres_air,phone,0.679
7,postgres_air,boarding_pass,0.052
8,postgres_air,flight,0.990
9,postgres_air,aircraft,0.816


# pg_stat_database
https://www.postgresql.org/docs/current/monitoring-stats.html#MONITORING-PG-STAT-DATABASE-VIEW


The ```pg_stat_database``` view will contain one row for each database in the cluster, plus one for shared objects, showing database-wide statistics.

In [4]:
query_pg_stat_database = """
select * 
from pg_stat_database 
order by datname
limit 100
"""

df = pd.read_sql_query(query_pg_stat_database, connection)
df



,datid,datname,numbackends,xact_commit,xact_rollback,blks_read,blks_hit,tup_returned,tup_fetched,tup_inserted,...,tup_deleted,conflicts,temp_files,temp_bytes,deadlocks,checksum_failures,checksum_last_failure,blk_read_time,blk_write_time,stats_reset
0,71456,airbases,6,2072118,3650,4521340291,4614741428,1042717829659,346049979,107716243,...,17334,0,116365,282108959674,0,0,None,8.117971e+07,16650.110,2022-07-24 13:22:40.397635+00:00
1,2242855,airbases-demo,1,153975,344,83454297,173269449,9574994072,37523416,44271,...,52,0,10017,115377312164,0,0,None,1.181759e+07,0.000,2023-06-06 11:53:01.986265+00:00
2,2232832,airbases-qa,0,164040,71,3368,6783612,98601395,1158782,0,...,0,0,0,0,0,0,None,9.966950e+02,0.000,2023-06-04 12:27:35.760749+00:00
3,16416,ariksa,0,3255052,4299,12236578,149146153,3271580451,30420725,36471769,...,1751,0,182,5273378816,0,0,None,2.113840e+05,6622.894,2022-03-24 18:14:50.881796+00:00
4,741903,bee_hero,1,772050,308,43182,918798945,12297226892,17721745,136694,...,43,0,1453,1187679056,0,0,None,4.339223e+03,0.000,2023-04-13 11:50:31.632107+00:00
5,742423,bee_hero_2,1,760275,105,3851,31565163,459766448,5339548,110,...,0,0,0,0,0,0,None,1.846498e+03,0.000,2023-04-13 12:10:39.320840+00:00
6,27603,books,1,2470419,145,5404,102558808,1498967185,17337390,33,...,18,0,0,0,0,0,None,2.377240e+03,0.000,2022-05-24 10:05:01.930212+00:00
7,669717,demo,0,1480757,58,1042361,102494170,1084704286,18433248,21810519,...,10,0,569,1494502662,0,0,None,5.591741e+04,1706.707,2023-01-13 06:32:27.650539+00:00
8,1279674,dev_lior_platform,0,301546,380,7026,14309879,254094495,2183345,3278,...,336,0,0,0,0,0,None,3.543105e+03,0.049,2023-05-23 18:56:25.588308+00:00
9,2238988,grafana_demo,0,150675,14,2769,6324731,93431503,1085608,110,...,0,0,0,0,0,0,None,1.171827e+03,0.000,2023-06-05 13:41:34.205259+00:00
